# 12.8 SQLDatabaseToolkit

In [6]:
from langchain.agents import create_sql_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-4o"
)

db = SQLDatabase.from_uri("sqlite:///movies.sqlite")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
)

agent.invoke("Give me the 5 directors that have the highest grossing films")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: directors, movies
Thought:The tables "directors" and "movies" are available. I should check the schema of these tables to understand their structure and find the relevant columns for the query. This will help me identify how to link directors to their movies and find the highest grossing films.

Action: sql_db_schema
Action Input: "directors, movies"
Observation: 
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INT

{'input': 'Give me the 5 directors that have the highest grossing films',
 'output': 'The 5 directors with the highest grossing films are:\n1. James Cameron with "Avatar" ($2,787,965,087)\n2. James Cameron with "Titanic" ($1,845,034,188)\n3. Joss Whedon with "The Avengers" ($1,519,557,910)\n4. Colin Trevorrow with "Jurassic World" ($1,513,528,810)\n5. James Wan with "Furious 7" ($1,506,249,360)'}

In [9]:
agent.invoke("Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: directors, movies
Thought:The relevant tables for this query are likely "movies" and "directors". I should check the schema of these tables to understand their structure and find the necessary columns for the query. 
Action: sql_db_schema
Action Input: "movies, directors"
Observation: 
CREATE TABLE directors (
	name TEXT, 
	id INTEGER, 
	gender INTEGER, 
	uid INTEGER, 
	department TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from directors table:
name	id	gender	uid	department
James Cameron	4762	2	2710	Directing
Gore Verbinski	4763	2	1704	Directing
Sam Mendes	4764	2	39	Directing
*/


CREATE TABLE movies (
	id INTEGER, 
	original_title VARCHAR, 
	budget INTEGER, 
	popularity INTEGER, 
	release_date TEXT, 
	revenue INTEGER, 
	title TEXT, 
	vote_average REAL, 
	vote_count INTEGER, 
	overview TEXT, 
	tagline TEXT, 
	uid INTEGER, 
	director_id INTEGER DEFAULT 0 NOT NULL, 
	PRIMARY KEY (id)
)

/*
3 rows 

{'input': 'Give me the movies that have the highest votes but the lowest budgets and give me the name of their directors also include their gross revenue.',
 'output': 'Here are some movies with the highest votes and lowest budgets, along with their directors and gross revenue:\n\n1. "Inception" directed by Christopher Nolan, Budget: $160,000,000, Votes: 13,752, Revenue: $825,532,764\n2. "The Dark Knight" directed by Christopher Nolan, Budget: $185,000,000, Votes: 12,002, Revenue: $1,004,558,444\n3. "Avatar" directed by James Cameron, Budget: $237,000,000, Votes: 11,800, Revenue: $2,787,965,087\n4. "The Avengers" directed by Joss Whedon, Budget: $220,000,000, Votes: 11,776, Revenue: $1,519,557,910\n5. "Deadpool" directed by Tim Miller, Budget: $58,000,000, Votes: 10,995, Revenue: $783,112,979\n6. "Interstellar" directed by Christopher Nolan, Budget: $165,000,000, Votes: 10,867, Revenue: $675,120,017\n7. "Django Unchained" directed by Quentin Tarantino, Budget: $100,000,000, Votes: 10,0